In [1]:
import numpy as np
import nashpy as nash
import itertools

First, a definition by John Nash: a **best response strategy** is one which maximizes the utility of a player given a known strategy of the other player.

Mathematically, in a 2-player normal form game $(A,B)$, a strategy $\sigma^*_A$ of player 1 is a best response to player 2's strategy $\sigma_B$ if

$$ \sigma^*_A = \operatorname{argmax}_{\sigma_A\in S_A}{\sigma_A^T A \sigma_B} $$

and similarly for player 2 and $\sigma^*_B$.

A pair of strategies $(\sigma_A, \sigma_B)$ is a **Nash equilibrium** if they are best responses to each other. We will outline the support enumeration algorithm for computing Nash equilibria. To start, the **support** of a strategy $\mathcal{S}(\sigma)$ is the set of strategies for which the strategy assigns positive probability.

In [2]:
def support(sigma, tol=1e-10):
    sigma = np.array(sigma)
    return np.where(sigma > tol)[0]

sigma = np.array([1/3, 1/2, 0, 0, 1/6])
support(sigma)

array([0, 1, 4])

Since finding Nash equilibria involves figuring out which strategies are best responses to the other player's strategy, we need a way to efficiently determine the whether the best response condition holds.

We claim (but not prove), that $x$ is a best response to $y$ if and only if for all $i$,
$$ x_i > 0 \implies (Ay)_i = \max\{(Ay)_k \} $$
To interpret this, note that for a player 2 strategy $y$, $Ay$ is the vector of payoffs for each strategy that player 1 could take.

In [3]:
def is_best_response(A, sigma_A, sigma_B):
    # set as np.arrays
    sigma_A, sigma_B = np.array(sigma_A), np.array(sigma_B)
    spt = support(sigma_A)
    best = True
    Ay = A @ sigma_B
    max_Ay = max(Ay)
    for i in spt:
        if Ay[i] != max_Ay:
            best = False
    return best
        
A = [[3, 0], [5, 1]]
sigma_A = [0, 1]
sigma_B = [1, 0]

is_best_response(A, sigma_A, sigma_B)

True

This leads to the support enumeration algorithm for computing Nash equilibria.

In [4]:
A = np.array([[160, 205, 44], [173, 180, 45], [201, 204, 50], [120, 207, 49]])
B = np.array([[2, 2, 0], [1, 0, 0], [3, 4, 1], [4, 1, 2]])

In [5]:
def indifference(payoff, support_u, support_v):
    n = len(payoff[0])
    M = np.zeros((len(support_u) + n - len(support_v), n), dtype=np.float64)
    # condition 1: best response condition for given support
    for i in range(len(support_u) - 1):
        M[i] = payoff[support_u[i]] - payoff[support_u[i+1]]
    # condition 2: ensure support is in S(v)
    complement_Sv = [j for j in range(n) if j not in support_v]
    for k, i in enumerate(range(len(support_u) - 1, len(support_u) + n - len(support_v) - 1)):
        M[i][complement_Sv[k]] = 1
    # condition 3: last row is all 1's to ensure solution is a probability vector
    M[-1] = np.ones(n)
    return M

indifference(A, [2,3], [0,1])

array([[81., -3.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  1.,  1.]])

In [6]:
indifference(B.T, [0,1], [2,3])

array([[ 0.,  1., -1.,  3.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  1.]])

In [7]:
def equilibrium_candidate(payoff, support_u, support_v):
    M = indifference(payoff, support_u, support_v)
    b = np.zeros(M.shape[1])
    b[-1] = 1
    # return np.linalg.inv(M) @ b
    return np.linalg.solve(M, b)
    
c = equilibrium_candidate(A, [2,3], [0,1])
d = equilibrium_candidate(B.T, [0,1], [2,3])
c, d

(array([0.03571429, 0.96428571, 0.        ]), array([0.  , 0.  , 0.75, 0.25]))

In [8]:
B.T @ d, A @ c

(array([3.25, 3.25, 1.25]),
 array([203.39285714, 179.75      , 203.89285714, 203.89285714]))

In [9]:
is_best_response(A, d, c) and is_best_response(B.T, c, d)

True

Finally, this generates candidates for Nash equlibria. To check if they really are, we need to check if they are best responses to one another. We can finally combine this into the support enumeration algorithm.

In [10]:
def get_powerset(n):
    """
    get powerset of n using itertools
    """
    return itertools.chain.from_iterable(
        itertools.combinations(range(n), r) for r in range(n + 1))

def equilibria(payoff_A, payoff_B):
    """
    find all Nash equilibria of 2-player normal form game via 
    support enumeration.
    """
    n, m = payoff_A.shape
    # create supports
    for support_u in (s for s in get_powerset(n) if len(s) > 0):
        for support_v in (s for s in get_powerset(m) if len(s) == len(support_u)):
            # get candidate for nash equilibrium
            try:
                candidate_A = equilibrium_candidate(payoff_A, support_u, support_v)
                candidate_B = equilibrium_candidate(payoff_B.T, support_v, support_u)
                # check if best response to one another
                if is_best_response(payoff_B.T, candidate_A, candidate_B) and \
                    is_best_response(payoff_A, candidate_B, candidate_A):
                        return [candidate_A, candidate_B]
            except:
                continue
    return "no Nash equilibria found"
                

equilibria(A, B)

[array([0.03571429, 0.96428571, 0.        ]), array([0.  , 0.  , 0.75, 0.25])]

In [11]:
# check answer with nashpy
game = nash.Game(A, B)
list(game.support_enumeration())

[(array([0.  , 0.  , 0.75, 0.25]),
  array([0.03571429, 0.96428571, 0.        ]))]